In [81]:
pip install scikit-image

In [82]:
import matplotlib.pyplot as plt

from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean

In [83]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import Flatten
from keras.layers import Dropout

In [84]:
#Importing cifar10 datasets
from keras.datasets import cifar10
(x_train, y_train),(x_test, y_test) =  cifar10.load_data()

In [86]:
y_train

array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

In [87]:
#Fetching only 1000 rows for each i.e. x_train, y_train, x_test, y_test
x_train = x_train[:1000]
x_test = x_test[:1000]
y_train = y_train[:1000]
y_test = y_test[:1000]

In [88]:
#the below function transform image size to Imagenet size i.e. 224x224x3
from skimage.util import img_as_ubyte
import skimage
import numpy as np

def convert_to_imagenet_size(images):
  tmp_images = []
  
  for image in images:
    image = skimage.transform.resize(image,(224,224),mode='constant')
    image = img_as_ubyte(image)
    tmp_images.append(image)
   
  return np.array(tmp_images)

In [89]:
#checking image shape before transforming 
print(x_train.shape)
print(x_test.shape)

(1000, 32, 32, 3)
(1000, 32, 32, 3)


In [90]:
x_train = convert_to_imagenet_size(x_train)
x_test = convert_to_imagenet_size(x_test)

In [91]:
#checking image shape after passing thru "convert_to_imagenet_size" function
print(x_train.shape)
print(x_test.shape)

(1000, 224, 224, 3)
(1000, 224, 224, 3)


In [92]:
#Performing Normalization
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train/255.
x_test = x_test/255.

In [93]:
#Applying one hot encoding on _train, y_test
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [94]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [95]:
#Model Creation
#Create a sequential model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11), strides=(4,4), padding='valid', activation='relu'))
# Pooling 
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid', activation='relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())


In [96]:
# 1st Dense Layer
model.add(Dense(4096, activation='relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096, activation='relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Dense Layer
model.add(Dense(1000, activation='relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# Output Layer
model.add(Dense(10, activation = 'softmax'))

model.summary()


Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 54, 54, 96)        34944     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_20 (Batc (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 17, 17, 256)       2973952   
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 8, 8, 256)         0         
_________________________________________________________________
batch_normalization_21 (Batc (None, 8, 8, 256)         1024      
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 6, 6, 384)        

In [97]:
#Compiling Model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [98]:
y_train.shape

(1000, 10)

In [99]:
#fitting Model
model.fit(x_train, y_train, batch_size=128, epochs=5,verbose=1, validation_split=0.2, shuffle=True)

Train on 800 samples, validate on 200 samples
Epoch 1/5
800/800 [==============================] - 76s 95ms/step - loss: 3.0564 - accuracy: 0.1775 - val_loss: 11.8141 - val_accuracy: 0.1150
Epoch 2/5
800/800 [==============================] - 76s 95ms/step - loss: 2.7873 - accuracy: 0.2587 - val_loss: 51.5046 - val_accuracy: 0.0850
Epoch 3/5
800/800 [==============================] - 76s 95ms/step - loss: 2.3602 - accuracy: 0.2725 - val_loss: 69.9181 - val_accuracy: 0.0850
Epoch 4/5
800/800 [==============================] - 76s 95ms/step - loss: 2.1224 - accuracy: 0.3350 - val_loss: 64.8057 - val_accuracy: 0.0850
Epoch 5/5
800/800 [==============================] - 76s 96ms/step - loss: 1.9193 - accuracy: 0.3500 - val_loss: 48.0894 - val_accuracy: 0.0850


In [100]:
#Evaluating Model
scores = model.evaluate(x_test, y_test, verbose=1)

1000/1000 [==============================] - 22s 22ms/step


In [101]:
print("Test Loss = ", scores[0])
print("Test Accuracy = ", scores[1])

Test Loss =  49.48676263427734
Test Accuracy =  0.10599999874830246
